In [1]:
!pip install -q -U keras-tuner

In [2]:
import tensorflow as tf
import keras_tuner as kt
print(tf.__version__)   # >2.8.0
print(kt.__version__)   # >1.1.2

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from tensorflow import keras 
from tensorflow.keras import layers
import keras_tuner as kt
import numpy as np
import IPython

2.8.0
1.1.2


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings, random
warnings.filterwarnings(action='ignore')

from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import StratifiedKFold

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
train = pd.read_csv('/content/drive/MyDrive/Likelion_Semi_project_03/data/train_occpy_pred_final.csv')
test = pd.read_csv('/content/drive/MyDrive/Likelion_Semi_project_03/data/test_occpy_pred_final.csv')

# Preprocessing

In [6]:
train.gender = train.gender.replace({'F' : 0, 'M' : 1})
train.car = train.car.replace({'N' : 0, 'Y' : 1})
train.reality = train.reality.replace({'N' : 0, 'Y' : 1})
train['age'] = train.DAYS_BIRTH.apply(lambda x : -x // 365)
train.DAYS_EMPLOYED = (-1) * train.DAYS_EMPLOYED 
train.loc[(train.DAYS_EMPLOYED < 0), 'DAYS_EMPLOYED'] = 0
train.begin_month = (-1) * train.begin_month

train = train.drop(['Unnamed: 0','DAYS_BIRTH'], axis = 1)
train.head(3)

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,credit,age
0,0,0,0,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,4709,0,0,0,Accountants,2.0,6.0,1.0,38
1,0,0,1,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,1540,0,0,1,Laborers,3.0,5.0,1.0,31
2,1,1,1,0,450000.0,Working,Higher education,Married,House / apartment,4434,0,1,0,Managers,2.0,22.0,2.0,52


In [7]:
test.gender = test.gender.replace({'F' : 0, 'M' : 1})
test.car = test.car.replace({'N' : 0, 'Y' : 1})
test.reality = test.reality.replace({'N' : 0, 'Y' : 1})
test['age'] = test.DAYS_BIRTH.apply(lambda x : -x // 365)
test.DAYS_EMPLOYED = (-1) * test.DAYS_EMPLOYED 
test.loc[(test.DAYS_EMPLOYED < 0), 'DAYS_EMPLOYED'] = 0
test.begin_month = (-1) * test.begin_month

test = test.drop(['Unnamed: 0','DAYS_BIRTH'], axis = 1)
test.head(3)

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,age
0,1,1,0,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,0,0,1,0,Security staff,2.0,60.0,60
1,0,0,1,0,135000.0,State servant,Higher education,Married,House / apartment,8671,0,1,0,Core staff,2.0,36.0,51
2,0,0,1,0,69372.0,Working,Secondary / secondary special,Married,House / apartment,217,1,1,0,Laborers,2.0,40.0,43


In [8]:
print(train.shape, test.shape)

(26457, 18) (10000, 17)


In [9]:
y = train.credit
X = train.drop(['credit'], axis = 1)

# 데이터 분할

In [10]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3)

# 데이터 인코딩

In [11]:
numeric_features = ['child_num', 'income_total', 'DAYS_EMPLOYED', 'family_size', 'begin_month', 'age']
numeric_transformer = StandardScaler()

categorical_features = ['income_type', 'edu_type', 'family_type', 'house_type','occyp_type']
categorical_transformer = OneHotEncoder(categories='auto', handle_unknown = 'ignore')


preprocessor = ColumnTransformer(
                transformers=[
                    ('num', numeric_transformer, numeric_features),
                    ('cat', categorical_transformer, categorical_features)
                ], remainder='passthrough'
                )

In [12]:
preprocessor.fit(X_train)
scaled_X_train = preprocessor.transform(X_train)
scaled_X_val = preprocessor.transform(X_val)

# Bayesian HPO

In [13]:
import tensorflow as tf
from tensorflow.keras import datasets, utils
from tensorflow.keras import models, layers, activations, initializers, losses, optimizers, metrics

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection, preprocessing

In [14]:
scaled_X_train.shape

(18519, 51)

In [15]:
train_label = utils.to_categorical(y_train) 
val_label = utils.to_categorical(y_val)

### Hyper_model

In [16]:
# Build the hyper-model
def build_hyper_model(hp):
    
    model = keras.Sequential()
    model.add(layers.Flatten())
        
    ### Hidden layer 만들기 (Search Space)
    # Tune the number of hidden layer (Choose an optimal value between 1~3)
    for layer_num in range(hp.Int('num_layers', min_value=1, max_value=3)):   # hp.int() : 후보 만드는 함수
        # Tune the number of perceptrons in a dense layer (Choose an optimal value between 32~512) 
        hp_units = hp.Int('units_' + str(layer_num), min_value=32, max_value=512, step=32) # 32:512 & step 32, all parameter names should be unique (we name the inner parameters 'units_' + str(i))  & 32, 512 포함
        hp_activations = hp.Choice('activation_' + str(layer_num), values=['relu', 'elu']) # hp.Choice : 문자열, 숫자열로 되있는 후보 중 하나 골라주는 것
        model.add(layers.Dense(units = hp_units, activation = hp_activations))

    # Output layer
    model.add(layers.Dense(3, activation='softmax')) # class 10 : 0~9
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
    
    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.CategoricalCrossentropy(),
                metrics = ['accuracy'])
    
    return model

In [17]:
# Select tuner and compile it
tuner = kt.BayesianOptimization(build_hyper_model,
                                objective = 'val_loss', # Hyper-params tuning을 위한 목적함수 설정 (metric to minimize or maximize)
                                max_trials = 10, # 서로 다른 Hyper-params 조합으로 시도할 총 Trial 횟수 설정
                                directory = 'test_prac_dir', # Path to the working directory
                                project_name = 'hyper_4') # Name to use as directory name for files saved by this Tuner
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
activation_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'elu'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


### Train

In [18]:
# Train the model
tuner.search(scaled_X_train, train_label, epochs=100, validation_data = (scaled_X_val, val_label))

Trial 10 Complete [00h 03m 22s]
val_loss: 0.8322784304618835

Best val_loss So Far: 0.805133581161499
Total elapsed time: 00h 35m 23s


In [19]:
tuner.results_summary(num_trials=3) 

Results summary
Results in test_prac_dir/hyper_4
Showing 3 best trials
Trial summary
Hyperparameters:
num_layers: 1
units_0: 512
activation_0: relu
learning_rate: 0.0001
units_1: 192
activation_1: relu
units_2: 64
activation_2: elu
Score: 0.805133581161499
Trial summary
Hyperparameters:
num_layers: 1
units_0: 512
activation_0: relu
learning_rate: 0.0001
units_1: 512
activation_1: relu
units_2: 384
activation_2: elu
Score: 0.8064849972724915
Trial summary
Hyperparameters:
num_layers: 1
units_0: 512
activation_0: relu
learning_rate: 0.0001
units_1: 32
activation_1: relu
units_2: 480
activation_2: elu
Score: 0.806498110294342


In [20]:
# Check the result 
top3_models = tuner.get_best_hyperparameters(num_trials=3)
for idx, model in enumerate(top3_models):
    print('Model performance rank :', idx)
    print(model.values)
    print()
    
best_hps = top3_models[0]

print("""
The hyperparameter search is complete. 
* Optimal # of layers : {}
* Optimal value of the learning-rate : {}""".format(best_hps.get('num_layers'), best_hps.get('learning_rate')))

for layer_num in range(best_hps.get('num_layers')):
    print('Layer {} - # of Perceptrons :'.format(layer_num), best_hps.get('units_' + str(layer_num)))
    print('Layer {} - Applied activation function :'.format(layer_num), best_hps.get('activation_' + str(layer_num)))

Model performance rank : 0
{'num_layers': 1, 'units_0': 512, 'activation_0': 'relu', 'learning_rate': 0.0001, 'units_1': 192, 'activation_1': 'relu', 'units_2': 64, 'activation_2': 'elu'}

Model performance rank : 1
{'num_layers': 1, 'units_0': 512, 'activation_0': 'relu', 'learning_rate': 0.0001, 'units_1': 512, 'activation_1': 'relu', 'units_2': 384, 'activation_2': 'elu'}

Model performance rank : 2
{'num_layers': 1, 'units_0': 512, 'activation_0': 'relu', 'learning_rate': 0.0001, 'units_1': 32, 'activation_1': 'relu', 'units_2': 480, 'activation_2': 'elu'}


The hyperparameter search is complete. 
* Optimal # of layers : 1
* Optimal value of the learning-rate : 0.0001
Layer 0 - # of Perceptrons : 512
Layer 0 - Applied activation function : relu


### Test

In [21]:
# Get the best model from trials
models = tuner.get_best_models(num_models=3) # Keras Sequential models
# 10개의 trials 중 가장 좋은 model 꺼내줌
models

In [22]:
top_model = models[0]
#top_model.summary()
top_model

In [23]:
scaled_X_test = preprocessor.transform(test)

In [24]:
y_pred = top_model.predict(scaled_X_test)
y_pred

array([[0.02344753, 0.01779467, 0.95875776],
       [0.19522214, 0.15804337, 0.6467345 ],
       [0.12336275, 0.17736068, 0.69927657],
       ...,
       [0.09256572, 0.08722784, 0.82020646],
       [0.1112118 , 0.17291237, 0.71587586],
       [0.09905817, 0.24483415, 0.65610766]], dtype=float32)

In [25]:
for trial in tuner.oracle.get_best_trials(num_trials=3):
    print('Trial-score is :', trial.score)
    print('Trial-directory(trial_id) is :', trial.trial_id)
    print()

Trial-score is : 0.805133581161499
Trial-directory(trial_id) is : 02

Trial-score is : 0.8064849972724915
Trial-directory(trial_id) is : 03

Trial-score is : 0.806498110294342
Trial-directory(trial_id) is : 05



In [26]:
submit = pd.read_csv('/content/drive/MyDrive/Likelion_Semi_project_03/data/sample_submission.csv')
submit.head()

,index,0,1,2
0,26457,0,0,0
1,26458,0,0,0
2,26459,0,0,0
3,26460,0,0,0
4,26461,0,0,0


In [27]:
submit.iloc[:, 1:] = y_pred
submit.head()

,index,0,1,2
0,26457,0.023448,0.017795,0.958758
1,26458,0.195222,0.158043,0.646734
2,26459,0.123363,0.177361,0.699277
3,26460,0.198897,0.099903,0.701199
4,26461,0.040693,0.282762,0.676545


In [28]:
submit.to_csv('/content/drive/MyDrive/Likelion_Semi_project_03/hpo_200_submission.csv',index=False)

In [29]:
top_model.evaluate(scaled_X_val, y_val)

ValueError: ignored

In [ ]:
results = top_model.evaluate(scaled_X_val, y_val)
print('Cross-entropy :', results[0])
print('Accuracy :', results[1])